In [1]:
import os
import pandas as pd
import numpy as np
from utils import to_21d_vec, get_beat_vector
import re

In [27]:
script_dir = os.getcwd()
score_rel_path = "../datasets/haydn_op20_harm/haydn_score_for_vec/1_iii_score_reduced_vector_pre.krn"
scorepath = os.path.join(script_dir, score_rel_path)


In [28]:
df = pd.read_csv(scorepath, sep="\t", header=None)
pd.set_option('display.max_rows', df.shape[0]+1)
#df.columns = ["harm", "root", "voice4", "voice3", "voice2", "voice1", "index", "beat", "measure", "key", "meter" ]
df.columns = ["harm", "voice4", "voice3", "voice2", "voice1", "index", "beat", "measure", "key", "meter" ]

In [29]:
#process dataframe, delete bars, score representations, and "rest slice"
df = df[~df['beat'].astype(str).str.startswith(('=','.','*'))]
df

,harm,voice4,voice3,voice2,voice1,index,beat,measure,key,meter
1,I,[4.AA-/,8A-/L,8c/L,[4.a-/,24,1,.,A-,M3/8
3,I,[4.AA-/,8c/J,8e-/J,[4.a-/,26,2,.,A-,M3/8
5,IVc,4.AA-/_,8d-\L,8f/L,4a-/],28,1,=2,A-,M3/8
6,IVc,4.AA-/_,8r,8e-/,4a-/],29,1.5,=2,A-,M3/8
7,IVc,4.AA-/_,8B-\J,8g/J,8dd-\,30,2,=2,A-,M3/8
9,IVc,8AA-/L],4A-/,[4.a-/,8cc\L,32,1,=3,A-,M3/8
10,IVc,8FF/,4A-/,[4.a-/,8dd-\,33,1.5,=3,A-,M3/8
11,IVc,8DD-/J,[8A-/,[4.a-/,8ff\J,34,2,=3,A-,M3/8
13,Ib,8CC/L,8A-/L],8a-/L],8ee-\L,36,1,=4,A-,M3/8
14,(V7b),8GG/,8E-/,8b-/,8r,37,1.5,=4,A-,M3/8


In [5]:
#s = re.sub('[^a-zA-Z#]+', '', df.loc[1,["voice4"]].values)
print(df.loc[33,["voice3"]].values)


['4e 4B']


In [34]:
prev_note_list = []
for index, row in df.iterrows():
    # for this onset slice 
    this_onset_vec = []
    
    ######################################### Process notes and onsets ########################################
    voices = ["voice4","voice3","voice2","voice1"]
    cur_note_tmp = []
    for v in voices:
        this_note = ''.join(row[[v]].values)
        if ' ' in this_note: #for a part has multiple voices
            this_note_list = this_note.split(" ")
            cur_note_tmp.extend(this_note_list)
        else:
            cur_note_tmp.append(this_note)
    
    cur_note_list = []
    for n in cur_note_tmp:
        note_name = re.sub('[^a-gA-G#-]+', '', n)
        if note_name:
            cur_note_list.append(note_name)
            
    cur_onset_list = list(set(cur_note_list) - set(prev_note_list))
    prev_note_list = cur_note_list
    
    print(cur_note_list)
    
    note_vec = to_21d_vec(cur_note_list)
    onset_vec = to_21d_vec(cur_onset_list)
    this_onset_vec.extend(note_vec)
    this_onset_vec.extend(onset_vec)
    
    ######################################### Process beat position ########################################
    beat_pos = ''.join(row[["beat"]].values)
    meter = ''.join(row[["meter"]].values)
    meter = meter.replace("M", "")
    beat_vec = get_beat_vector(beat_pos, meter)
    this_onset_vec.extend(beat_vec)
    
    ######################################### Process chord position ########################################
    harm = ''.join(row[["harm"]].values)
    key = ''.join(row[["key"]].values)
    #########################################################################################################
    #process notes
    #row["voice4"] = re.sub('[^a-zA-Z#-]+', '', row[["voice4"]].values)
    #row["voice3"] = re.sub('[^a-zA-Z#-]+', '', row[["voice3"]].values)
    #row["voice2"] = re.sub('[^a-zA-Z#-]+', '', row[["voice2"]].values)
    #row["voice1"] = re.sub('[^a-zA-Z#-]+', '', row[["voice1"]].values)
#df

['AA-', 'A-', 'c', 'a-']
['AA-', 'c', 'e-', 'a-']
['AA-', 'd-', 'f', 'a-']
['AA-', 'e-', 'a-']
['AA-', 'B-', 'g', 'dd-']
['AA-', 'A-', 'a-', 'cc']
['FF', 'A-', 'a-', 'dd-']
['DD-', 'A-', 'a-', 'ff']
['CC', 'A-', 'a-', 'ee-']
['GG', 'E-', 'b-']
['AA-', 'E-', 'e-', 'cc']
['DD-', 'D-', 'f', 'b-']
['DD-', 'd-', 'f', 'b-']
['EE-']
['EE-', 'B-', 'd-', 'g']
['AA-', 'A-', 'c', 'a-']
['AA-', 'd-', 'f', 'a-']
['AA-', 'B-', 'e-', 'a-']
['AA-', 'A-', 'd-', 'a-']
['AA-', 'A-', 'B-', 'd-']
['AA-', 'A-', 'c', 'a-']
['C', 'A-', 'e-', 'a-']
['D-', 'A-', 'f', 'a-']
['A-', 'e-', 'a-']
['BB-', 'e-', 'g', 'dd-']
['AA-', 'e-', 'a-', 'cc']
['F', 'd-', 'a-', 'dd-']
['D-', 'd-', 'a-', 'ff']
['C', 'e-', 'a-', 'ee-']
['G', 'e-', 'b-']
['A-', 'e-', 'e-', 'cc']
['D-', 'd-', 'f', 'b-']
['BB-', 'd-', 'd-', 'ff']
['D', 'd', 'f', 'a-']
['E-', 'e-', 'B-', 'g']
['E-', 'e-', 'g', 'ee-']
['F', 'e-', 'a-', 'dd']
['G', 'e-', 'b-', 'dd-']
['e-', 'e-', 'cc']
['G', 'B-', 'g', 'ee']
['F', 'A-', 'a-', 'ff']
['b-']
['D', 'F', 'b-